In [10]:
from shapely.geometry import box
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from shapely.ops import unary_union
from fiona.crs import from_epsg
import os

In [18]:
from geopy.distance import geodesic

def meters_to_degrees(meters, latitude):
    # Calculate the distance per degree at the specified latitude
    meters_per_degree = geodesic((latitude, 0), (latitude + 1, 0)).meters

    # Convert meters to degrees
    degrees = meters / meters_per_degree

    return degrees

In [19]:
def generate_bounding_boxes(tiff_path, box_size):
    """
    Generate fixed-sized square bounding boxes inside a given TIFF file.

    Parameters:
    - tiff_path: Path to the TIFF file
    - box_size: Size of the square bounding boxes

    Returns:
    - List of Shapely Polygon objects representing the bounding boxes
    """
    # Open the TIFF file and get its bounds
    with rasterio.open(tiff_path) as src:
        bounds = box(*src.bounds)
    avg_lat = (bounds.bounds[1] + bounds.bounds[2]) / 2
    box_size = meters_to_degrees(box_size, avg_lat)
    # Calculate the number of boxes in the x and y directions
    num_boxes_x = int((bounds.bounds[2] - bounds.bounds[0]) // box_size)
    num_boxes_y = int((bounds.bounds[3] - bounds.bounds[1]) // box_size)

    # Generate the bounding boxes
    bounding_boxes = []
    for i in range(num_boxes_x):
        for j in range(num_boxes_y):
            # Calculate the coordinates of the box
            min_x = bounds.bounds[0] + i * box_size
            min_y = bounds.bounds[1] + j * box_size
            max_x = min_x + box_size
            max_y = min_y + box_size

            # Create a Shapely box
            box_geometry = box(min_x, min_y, max_x, max_y)

            # Check if the box is completely within the TIFF bounds
            if bounds.contains(box_geometry):
                bounding_boxes.append(box_geometry)

    return bounding_boxes


tiff_path = '../TropicalTreeCover_NorthernQLD-0000000000-0000000000.tif'  
box_size = 5120

# Generate bounding boxes
bounding_boxes = generate_bounding_boxes(tiff_path, box_size)

# Save the bounding boxes to a GeoJSON file
target_crs = from_epsg(4326)  # Assuming WGS84 for the example
gdf = gpd.GeoDataFrame(geometry=bounding_boxes, crs=target_crs)
gdf['file_path'] = os.path.abspath(tiff_path)
gdf.to_file('bounding_boxes.geojson', driver='GeoJSON')


In [20]:
gdf

,geometry,file_path
0,"POLYGON ((121.59641 -14.96758, 121.59641 -14.9...",/root/sentinel-tree-segmentation/TropicalTreeC...
1,"POLYGON ((121.59641 -14.92160, 121.59641 -14.8...",/root/sentinel-tree-segmentation/TropicalTreeC...
2,"POLYGON ((121.59641 -14.87562, 121.59641 -14.8...",/root/sentinel-tree-segmentation/TropicalTreeC...
3,"POLYGON ((121.59641 -14.82965, 121.59641 -14.7...",/root/sentinel-tree-segmentation/TropicalTreeC...
4,"POLYGON ((121.59641 -14.78367, 121.59641 -14.7...",/root/sentinel-tree-segmentation/TropicalTreeC...
...,...,...
16370,"POLYGON ((127.57362 -9.45016, 127.57362 -9.404...",/root/sentinel-tree-segmentation/TropicalTreeC...
16371,"POLYGON ((127.57362 -9.40418, 127.57362 -9.358...",/root/sentinel-tree-segmentation/TropicalTreeC...
16372,"POLYGON ((127.57362 -9.35820, 127.57362 -9.312...",/root/sentinel-tree-segmentation/TropicalTreeC...
16373,"POLYGON ((127.57362 -9.31222, 127.57362 -9.266...",/root/sentinel-tree-segmentation/TropicalTreeC...
